### Importação de bibliotecas

In [ ]:
import pandas as pd
import pathlib
import smtplib
import os
from dotenv import load_dotenv
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders


load_dotenv()

### Importação do arquivo fonte de dados

In [ ]:
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
email = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding = 'latin1', sep=';')


vendas = vendas.merge(lojas, on='ID Loja') #inserir o nome da loja na tabela vendas, conforme o ID da Loja.


vendas.info()
lojas.info()
email.info()

display(email)

### Criação das tabelas de cada loja

In [ ]:
dic_lojas = {}

for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :] # os ':" representam que quero todas as colunas que as linhas tenham a loja

dict_keys = dic_lojas.keys()

print(dict_keys)

### Salvar os arquivos individuais

In [ ]:
# identifica se a pasta existe, caso não, cria a pasta

caminho = pathlib.Path(r'Backup Arquivos Lojas')

data_salvar = vendas['Data'].max()

arquivo_pasta = caminho.iterdir()
nome_arquivos = [arquivo.name for arquivo in arquivo_pasta]

for loja in dic_lojas:
    if loja not in nome_arquivos:
        pasta = caminho / loja
        pasta.mkdir()

# salva o arquivo dentro da pasta com o nome da loja

arquivo = '{}_{}_{}.xlsx'.format(data_salvar.month, data_salvar.day, loja) # gerou o nome do arquivo
local = caminho / loja / arquivo # determina o local para salvar o arquivo 

dic_lojas[loja].to_excel(local) # criou os arquivos de cada loja, dentro das pastas de cada loja


### Estabelecer as metas

In [ ]:
meta_faturamento_ano = 1650000
meta_faturamento_dia = 1000
meta_produtos_ano =120  # (quantos produtos diferentes foram vendidos naquele período)
meta_produtos_dia =4
meta_ticket_ano = 500
meta_ticket_dia = 500

### Determinar os indicadores para cada loja e Enviar Email com os One Page para cada Loja

In [ ]:
for loja in dic_lojas:
    gerente = email.loc[email['Loja']==loja,'Gerente'].values[0]
    e_mail = email.loc[email['Loja']==loja,"E-mail"].values[0]
    dados_loja = dic_lojas[loja]
    venda_loja_ano = dados_loja['Valor Final'].sum()
    venda_loja_dia = dados_loja.loc[dados_loja['Data']==data_salvar]['Valor Final'].sum()
    produto_loja_ano = len(dados_loja['Produto'].unique())
    produto_loja_dia = len(dados_loja.loc[dados_loja['Data']==data_salvar]['Produto'].unique())
    valor_por_venda = dados_loja.groupby('Código Venda').sum(numeric_only=True)
    valor_por_venda_dia = dados_loja.loc[dados_loja['Data']==data_salvar].groupby('Código Venda').sum(numeric_only=True)
    ticket_loja_ano = valor_por_venda['Valor Final'].mean()
    ticket_loja_dia = valor_por_venda_dia['Valor Final'].mean()
    
    if venda_loja_ano >= meta_faturamento_ano:
        cor_fat_ano = "green"
    else:
        cor_fat_ano = "red"
    
    if venda_loja_dia >= meta_faturamento_dia:
        cor_fat_dia = "green"
    else:
        cor_fat_dia = "red"
    
    if ticket_loja_ano >= meta_ticket_ano:
        cor_tichet_ano = "green" 
    else:
        cor_ticket_ano = "red"
    
    if ticket_loja_dia >= meta_ticket_dia:
        cor_ticket_dia = "green"
    else: 
        cor_ticket_dia = "red"
    
    if produto_loja_ano >= meta_produtos_ano:
        cor_produto_ano = "green"
    else: 
        cor_produto_ano = "red"
    
    if produto_loja_dia >= meta_produtos_dia:
        cor_produto_dia = "green" 
    else: 
        cor_produto_dia = "red"


## Enviando os emails

    SENHA_GMAIL=os.getenv('SENHA_GMAIL')

    def enviar_email():
        msg = MIMEMultipart()
        msg['Subject'] = f"One Page - {data_salvar.day}/{data_salvar.month} - Loja: {loja}"
        msg['From'] ='Insira o E-Mail de Envio' #<-- Insira um email válido
        msg['To'] = e_mail
        # msg['Cc']
        # msg['BCo']

        mensagem = f"""
        Bom dia, {gerente},
        
        <p>O Resultado na data de <strong>{data_salvar.day} / {data_salvar.month}</strong> da <strong>Loja {loja}</strong> foi de:</p>
    
        <table>
        <tr>
            <th>Indicador</th>
            <th style="text-align: center">Valor do Dia</th>
            <th style="text-align: center">Meta do Dia</th>
            <th style="text-align: center">Cenário do Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center"> R$ {venda_loja_dia: .2f}</td>
            <td style="text-align: center">R$ {meta_faturamento_dia: .2f}</td>
            <td style="text-align: center"><font color = {cor_fat_dia}>◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {ticket_loja_dia: .2f}</td>
            <td style="text-align: center">R$ {meta_ticket_dia: .2f}</td>
            <td style="text-align: center"><font color = {cor_ticket_dia}>◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{produto_loja_dia}</td>
            <td style="text-align: center">{meta_produtos_dia}</td>
            <td style="text-align: center"><font color = {cor_produto_dia}>◙</font></td>
        </tr>
        </table>

        <br>

        <table>
        <tr>
            <th>Indicador</th>
            <th style="text-align: center">Valor do Ano</th>
            <th style="text-align: center">Meta do Ano</th>
            <th style="text-align: center">Cenário do Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td style="text-align: center"> R$ {venda_loja_ano: .2f}</td>
            <td style="text-align: center">R$ {meta_faturamento_ano: .2f}</td>
            <td style="text-align: center"><font color = {cor_fat_ano}>◙</font></td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R$ {ticket_loja_ano: .2f}</td>
            <td style="text-align: center">R$ {meta_ticket_ano: .2f}</td>
            <td style="text-align: center"><font color = {cor_tichet_ano}>◙</font></td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td style="text-align: center">{produto_loja_ano}</td>
            <td style="text-align: center">{meta_produtos_ano}</td>
            <td style="text-align: center"><font color = {cor_produto_ano}>◙</font></td>
        </tr>
        </table>
        <br>

        <p> Segue anexo a planilha detalhada para análise</p>

        """
               
        msg.attach(MIMEText(mensagem, "html"))

        pasta_base = pathlib.Path.cwd() / 'Backup Arquivos Lojas' 
        caminho_arquivo = pasta_base / loja / '{}_{}_{}.xlsx'.format(data_salvar.month, data_salvar.day, loja)
        
    
        try:
            with open(caminho_arquivo, "rb") as arquivo:
                part = MIMEBase("application", "octet-stream")
                part.set_payload(arquivo.read())
            
            encoders.encode_base64(part)
            part.add_header(
                "Content-Disposition",
                f"attachment; filename={caminho_arquivo.name}",
            )
            msg.attach(part)
        except Exception as e:
            print(f"Erro ao encontrar ou ler o anexo: {e}")
        
        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as servidor:
                servidor.starttls()
                servidor.login(msg['From'],SENHA_GMAIL)
                servidor.send_message(msg)
                print("E-mail enviado com sucesso!")
        except Exception as e:
            print(f"Erro ao enviar: {e}")
    
    
    enviar_email()


### Criar ranking das lojas

In [16]:
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_ano)

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(data_salvar.month, data_salvar.day)
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


vendas_dia = vendas.loc[vendas['Data']==data_salvar, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)
display(faturamento_lojas_dia)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(data_salvar.month, data_salvar.day)
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))



,Loja,Valor Final
Loja,,
Iguatemi Campinas,Iguatemi CampinasIguatemi CampinasIguatemi Cam...,1762419
Shopping Vila Velha,Shopping Vila VelhaShopping Vila VelhaShopping...,1731167
Bourbon Shopping SP,Bourbon Shopping SPBourbon Shopping SPBourbon ...,1726110
Rio Mar Recife,Rio Mar RecifeRio Mar RecifeRio Mar RecifeRio ...,1722766
Shopping SP Market,Shopping SP MarketShopping SP MarketShopping S...,1721763
Palladium Shopping Curitiba,Palladium Shopping CuritibaPalladium Shopping ...,1721120
Norte Shopping,Norte ShoppingNorte ShoppingNorte ShoppingNort...,1711968
Ribeirão Shopping,Ribeirão ShoppingRibeirão ShoppingRibeirão Sho...,1707122
Iguatemi Esplanada,Iguatemi EsplanadaIguatemi EsplanadaIguatemi E...,1699681


C:\Users\Andre\AppData\Local\Temp\ipykernel_16880\1536350730.py:6: UserWarning: Cell contents too long (69836), truncated to 32767 characters
  faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))
C:\Users\Andre\AppData\Local\Temp\ipykernel_16880\1536350730.py:6: UserWarning: Cell contents too long (80446), truncated to 32767 characters
  faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))
C:\Users\Andre\AppData\Local\Temp\ipykernel_16880\1536350730.py:6: UserWarning: Cell contents too long (78204), truncated to 32767 characters
  faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))
C:\Users\Andre\AppData\Local\Temp\ipykernel_16880\1536350730.py:6: UserWarning: Cell contents too long (57386), truncated to 32767 characters
  faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))
C:\Users\Andre\AppData\Local\Temp\ipykernel_16880\1536350730.py:6: UserWarning: Cell contents too lo

,Loja,Valor Final
Loja,,
Salvador Shopping,Salvador ShoppingSalvador Shopping,3950
Novo Shopping Ribeirão Preto,Novo Shopping Ribeirão PretoNovo Shopping Ribe...,3400
Center Shopping Uberlândia,Center Shopping UberlândiaCenter Shopping Uber...,2651
Shopping Eldorado,Shopping EldoradoShopping Eldorado,2391
Shopping Center Interlagos,Shopping Center InterlagosShopping Center Inte...,1582
Shopping Recife,Shopping RecifeShopping RecifeShopping Recife,1366
Norte Shopping,Norte ShoppingNorte Shopping,1259
Shopping União de Osasco,Shopping União de OsascoShopping União de Osas...,1207
Shopping Vila Velha,Shopping Vila VelhaShopping Vila VelhaShopping...,937


### Enviar email para diretoria com o ranking das lojas

In [ ]:
def enviar_email():
        msg = MIMEMultipart()
        msg['Subject'] = f"Ranking de Lojas - {data_salvar.day}/{data_salvar.month}"
        msg['From'] ='Insira o E-Mail de Envio' #<-- Insira um email válido
        msg['To'] = emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0]
        # msg['Cc']
        # msg['BCo']

        mensagem = f"""
            Prezados, bom dia

            Melhor loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[0]} com Faturamento R${float(faturamento_lojas_dia.iloc[0, 1]):.2f}
            Pior loja do Dia em Faturamento: Loja {faturamento_lojas_dia.index[-1]} com Faturamento R${float(faturamento_lojas_dia.iloc[-1, 1]):.2f}

            Melhor loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[0]} com Faturamento R${float(faturamento_lojas_ano.iloc[0, 1]):.2f}
            Pior loja do Ano em Faturamento: Loja {faturamento_lojas_ano.index[-1]} com Faturamento R${float(faturamento_lojas_ano.iloc[-1, 1]):.2f}

            Segue em anexo os rankings do ano e do dia de todas as lojas.

            Qualquer dúvida estou à disposição.

            Att.,
            
            """
            
               
        msg.attach(MIMEText(mensagem, "html"))

        # Anexo 1: Ranking Anual
        caminho_anual = pasta_base / f'{data_salvar.month}_{data_salvar.day}_Ranking Anual.xlsx'
        with open(caminho_anual, "rb") as arquivo:
            part1 = MIMEBase("application", "octet-stream")
            part1.set_payload(arquivo.read())
            encoders.encode_base64(part1)
            part1.add_header("Content-Disposition", f"attachment; filename={caminual_anual.name}")
            msg.attach(part1)

        # Anexo 2: Ranking Dia
        caminho_dia = pasta_base / f'{data_salvar.month}_{data_salvar.day}_Ranking Dia.xlsx'
        with open(caminho_dia, "rb") as arquivo:
            part2 = MIMEBase("application", "octet-stream")
            part2.set_payload(arquivo.read())
            encoders.encode_base64(part2)
            part2.add_header("Content-Disposition", f"attachment; filename={caminho_dia.name}")
            msg.attach(part2)
    
        try:
            with open(caminho_arquivo, "rb") as arquivo:
                part = MIMEBase("application", "octet-stream")
                part.set_payload(arquivo.read())
            
            encoders.encode_base64(part)
            part.add_header(
                "Content-Disposition",
                f"attachment; filename={caminho_arquivo.name}",
            )
            msg.attach(part)
        except Exception as e:
            print(f"Erro ao encontrar ou ler o anexo: {e}")
        
        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as servidor:
                servidor.starttls()
                servidor.login(msg['From'],SENHA_GMAIL)
                servidor.send_message(msg)
                print("E-mail enviado com sucesso!")
        except Exception as e:
            print(f"Erro ao enviar: {e}")
    
    
enviar_email()

E-mail enviado com sucesso!
